<a href="https://colab.research.google.com/github/the-Quert/inlpfun/blob/master/Lab/Demo/BERT/bert_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Mount GDrive

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Install pip packages

In [41]:
!pip uninstall tensorflow-gpu
!pip install scikit-learn
!pip install tensorflow-gpu==1.15.0
!pip install tensorflow-hub
!pip install bert-tensorflow

Streaming output truncated to the last 5000 lines.
    /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/com_github_grpc_grpc/include/grpc/support/atm_gcc_atomic.h
    /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/com_github_grpc_grpc/include/grpc/support/atm_gcc_sync.h
    /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/com_github_grpc_grpc/include/grpc/support/atm_windows.h
    /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/com_github_grpc_grpc/include/grpc/support/cpu.h
    /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/com_github_grpc_grpc/include/grpc/support/log.h
    /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/com_github_grpc_grpc/include/grpc/support/log_windows.h
    /usr/local/lib/python3.6/dist-packages/tensorflow/include/external/com_github_grpc_grpc/include/grpc/support/port_platform.h
    /usr/local/lib/python3.6/dist-packages/tensorflow/include/external

#### Import general modules

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [43]:
print(tf.__version__)

1.15.0


#### Tensorflow v1 modules rename to get compatible with Tensorflow 2.x

In [0]:
# import tensorflow.compat.v1 as tf
import tensorflow as tf

In [0]:
tf.gfile = tf.io.gfile
tf.train.Optimizer = tf.compat.v1.train.Optimizer
tf.flags = tf.compat.v1.flags
tf.logging = tf.compat.v1.logging
tf.get_variable = tf.compat.v1.get_variable
tf.truncated_normal_initializer = tf.compat.v1.truncated_normal_initializer
tf.variable_scope = tf.compat.v1.variable_scope

#### Import BERT module

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
# See modification from https://github.com/the-Quert/inlpfun/tree/master/Lab/demo/bert/modification_for_colab

#### Processing with dataset building.

In [0]:
from tensorflow import keras
import os
import re

# Load files from DIR in dataframe.
def load_DIR_data(dir):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(dir):
        with tf.gfile.GFile(os.path.join(dir, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(dir):
    pos_df = load_DIR_data(os.path.join(dir, "pos"))
    neg_df = load_DIR_data(os.path.join(dir, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop = True)

# Download and process the dataset files.
def download_and_load_datasets(force_download = False):
    dataset = tf.keras.utils.get_file(fname = "aclImdb.tar.gz", origin = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", extract = True)
    train_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "test"))
    return train_df, test_df

In [0]:
train, test = download_and_load_datasets()

In [0]:
train = train.sample(5000)
test = test.sample(5000)

In [51]:
train.columns

Index(['sentence', 'sentiment', 'polarity'], dtype='object')

In [0]:
DATA_COLUMN = "sentence"
LABEL_COLUMN = "polarity"
# List of labels
label_list = [0, 1]

In [0]:
# Use BERT's run_classifier to create sample from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [56]:
# Path to uncased version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [57]:
# Deploy testing
tokenizer.tokenize("Here is an example of using BERT tokenizer")

['here', 'is', 'an', 'example', 'of', 'using', 'bert', 'token', '##izer']

In [58]:
# Set maximum of sequence length
MAX_SEQ_LEN = 128
# Convert train and test features to InputFeatures that BERT understands
# convert_examples_to_features, convert_single_example
# Link to run_classifier.convert_example_to_features: htttps://github.com/the-Quert/inlpfun/tree/master/bert/run_classifier#L482-511
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LEN, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LEN, tokenizer)

INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


#### Creating a model

In [0]:
# Create a classification model with "run_classifier_with_tfhub.create_model"

def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
# run_classifier_with_tfhub.model_fn_builder
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size
# Hyperparameters are copied from GitHub/bert/MRPC
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
OUTPUT_DIR = "./"

In [0]:
# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# run_classifier_with_tfhub.main()
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [81]:
# run_classifier_with_tfhub.main()
# Migrate from TPU to GPU
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd1731472e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd1731472e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# run_classifier_with_tfhub.main()
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LEN,
    is_training=True,
    drop_remainder=False)

#### Beginning Training

In [89]:
print(f'Beginning Training...')
current_time = datetime.now()
estimator.train(input_fn = train_input_fn, max_steps = num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training...
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


...

INFO:tensorflow:Saving checkpoints for 468 into ./model.ckpt.


INFO:tensorflow:Saving checkpoints for 468 into ./model.ckpt.


INFO:tensorflow:Loss for final step: 0.017658286.


INFO:tensorflow:Loss for final step: 0.017658286.


Training took time  0:13:26.958873


In [0]:
# run_classifier.input_fn_builder
test_input_fn = run_classifier.input_fn_builder(
    features = test_features,
    seq_length = MAX_SEQ_LEN,
    is_training = False,
    drop_remainder = False
)

In [91]:
estimator.evaluate(input_fn = test_input_fn, steps = None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 468: auc = 0.86175936, eval_accuracy = 0.8616, f1_score = 0.8615999, false_negatives = 380.0, false_positives = 312.0, global_step = 468, loss = 0.50209904, precision = 0.8734793, recall = 0.8500395, true_negatives = 2154.0, true_positives = 2154.0


INFO:tensorflow:Saving dict for global step 468: auc = 0.86175936, eval_accuracy = 0.8616, f1_score = 0.8615999, false_negatives = 380.0, false_positives = 312.0, global_step = 468, loss = 0.50209904, precision = 0.8734793, recall = 0.8500395, true_negatives = 2154.0, true_positives = 2154.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 468: ./model.ckpt-468


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 468: ./model.ckpt-468


{'auc': 0.86175936,
 'eval_accuracy': 0.8616,
 'f1_score': 0.8615999,
 'false_negatives': 380.0,
 'false_positives': 312.0,
 'global_step': 468,
 'loss': 0.50209904,
 'precision': 0.8734793,
 'recall': 0.8500395,
 'true_negatives': 2154.0,
 'true_positives': 2154.0}

In [0]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LEN, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LEN, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
def getSinglePrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LEN, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LEN, is_training=False, drop_remainder=False)
  predictions = estimator.predict(input_fn=predict_input_fn, yield_single_examples=False)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

#### Testing Prediction

In [0]:
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

In [99]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from ./model.ckpt-468


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [100]:
predictions

[('That movie was absolutely awful',
  array([-2.6179824e-03, -5.9466782e+00], dtype=float32),
  'Negative'),
 ('The acting was a bit lacking',
  array([-0.00779125, -4.8586435 ], dtype=float32),
  'Negative'),
 ('The film was creative and surprising',
  array([-5.5701632e+00, -3.8171776e-03], dtype=float32),
  'Positive'),
 ('Absolutely fantastic!',
  array([-5.045494  , -0.00645907], dtype=float32),
  'Positive')]